In [ ]:
# import necessary layers  
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras import Model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


input = Input(shape =(256,256,3))
# 1st Conv Block

x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(input)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block

x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block

x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block

x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

# 5th Conv block

x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
x = Dropout(0.2)(x)
# Fully connected layers

x = Flatten()(x)
x = Dense(units = 256, activation ='relu')(x)
x = Dense(units = 128, activation ='relu')(x)
output = Dense(units = 2, activation ='softmax')(x)
# creating the model

model = Model (inputs=input, outputs =output)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [ ]:
train_dir = 'dataset/train_images'
val_dir = 'dataset/val_images'
test_dir = 'dataset/test_images'
train_dir2 = 'train'

#datagen = ImageDataGenerator(1./255)
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=64,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'training'
)
val_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=64,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'validation'
)
print(train_generator.class_names)

In [ ]:
history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=40)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image

img = image.load_img('dataset/test_images/fake/df2.jpg', target_size=(256,256))
img_1 = Image.open('dataset/test_images/fake/df2.jpg')
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

img2 = image.load_img('dataset/test_images/real/real00240.jpg', target_size=(256,256))
img_2 = Image.open('dataset/test_images/real/real00240.jpg')
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis = 0)

classes = ['fake','real']

#print(classes[np.argmax(model.predict(img))])
#print(classes[np.argmax(model.predict(img2))])

plt.imshow(img_1)
plt.title(classes[np.argmax(model.predict(img))])

In [ ]:
plt.imshow(img_2)
plt.title(classes[np.argmax(model.predict(img2))])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save('model/vgg16/vgg16_40_dropout_2.h5')

In [ ]:
#pip install pillow

In [ ]:
""""from tensorflow.keras.models import load_model

model = load_model('model/vgg16/vgg16_4.h5')

from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])""""

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model/vgg16/vgg16_4.h5')
model.summary()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img('dataset/test_images/fake/df2.jpg',target_size=(256,256))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
img /= 255.

img.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img[0])

In [ ]:
print(len(model.layers))

In [ ]:
top4_layers = [x.output for x in model.layers[:20]]
top4_model = Model(inputs=model.input,outputs=top4_layers)

pre = top4_model.predict(img)

In [ ]:
f1 = pre[0]
print(f1.shape)
plt.matshow(f1[0,:,:,22],cmap='viridis')
plt.show()

In [ ]:
import tensorflow.keras as keras

# These are the names of the layers, so can have them as part of our plot
layer_names = []
for layer in model.layers[:20]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, pre):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()

In [ ]:

test_dir = 'test'

test_generator = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    batch_size=64,
    image_size=(256,256),
    seed=123,
    color_mode='rgb'
)

"""dataGenerator = ImageDataGenerator(rescale=1./255)
test_gen = dataGenerator.flow_from_directory(
            val_dir,
            target_size=(256,256),
            batch_size=64,
            class_mode='categorical'
)"""

score = model.evaluate(test_generator)

In [ ]:
print(score)